In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
import wrds
from scipy import stats
os.chdir('/Users/yiyujie/Desktop/program/Quantitative Asset Management')

In [122]:
conn = wrds.Connection(wrds_username='aspyyj612')

Loading library list...
Done


## Q1
Construct the equal-weighted bond market return, value-weighted bond market return, and lagged total bond market capitalization using CRSP Bond data 1. Your output should be from January 1926 to December 2023, at a monthly frequency.

In [5]:
bonds = conn.raw_sql("""
                      select kycrspid, mcaldt, tmretnua, tmtotout
                      from crspq.tfz_mth
                      """)
bonds['mcaldt']   = pd.DataFrame(bonds[['mcaldt']].values.astype('datetime64[ns]')) + MonthEnd(0)

In [6]:
def PS2_Q1(CRSP_Bonds):
    CRSP_Bonds = CRSP_Bonds.rename(columns={"mcaldt":"date","tmretnua":"ret","tmtotout":"me","kycrspid":"idCRSP"}).copy()
    CRSP_Bonds.sort_values(by='date', inplace=True)
    equal_weighted_return = CRSP_Bonds.groupby([CRSP_Bonds['date'].dt.year, CRSP_Bonds['date'].dt.month]).apply(lambda x: x['ret'].mean())
    CRSP_Bonds['lagged_me'] = CRSP_Bonds.groupby('idCRSP')['me'].shift(1) * 1e-3
    value_weighted_return = CRSP_Bonds.groupby([CRSP_Bonds['date'].dt.year, CRSP_Bonds['date'].dt.month]).apply(lambda x: (x['ret'] * x['lagged_me']).sum() / x['lagged_me'].sum())
    lagged_total_market_cap = CRSP_Bonds.groupby([CRSP_Bonds['date'].dt.year, CRSP_Bonds['date'].dt.month]).apply(lambda x: x['lagged_me'].sum())
    # Filter the results to the desired date range
    result = pd.DataFrame({
        'Year': equal_weighted_return.index.get_level_values(0),
        'Month': equal_weighted_return.index.get_level_values(1),
        'Bond_lag_MV' : lagged_total_market_cap.values,
        'Bond_Ew_Ret' : equal_weighted_return.values,
        'Bond_Vw_Ret' : value_weighted_return.values
    })
    result = result[(result.Year>=1926)&(result.Year<=2023)].reset_index(drop=True)
    return result


Monthly_CRSP_Bonds = PS2_Q1(bonds[bonds.tmtotout.notna()])
Monthly_CRSP_Bonds

/var/folders/l_/m6g3497s5j77bqb9jr7506tc0000gn/T/ipykernel_16461/381320136.py:6: RuntimeWarning: invalid value encountered in scalar divide
  value_weighted_return = CRSP_Bonds.groupby([CRSP_Bonds['date'].dt.year, CRSP_Bonds['date'].dt.month]).apply(lambda x: (x['ret'] * x['lagged_me']).sum() / x['lagged_me'].sum())


,Year,Month,Bond_lag_MV,Bond_Ew_Ret,Bond_Vw_Ret
0,1926,1,19.502,0.005101,0.006829
1,1926,2,19.502,0.003621,0.003844
2,1926,3,18.884,0.003812,0.003588
3,1926,4,19.231,0.004014,0.006486
4,1926,5,19.227,0.002146,0.002629
...,...,...,...,...,...
1171,2023,8,22692.061,-0.003755,-0.003610
1172,2023,9,23294.014,-0.017121,-0.015735
1173,2023,10,23050.525,-0.010922,-0.010214
1174,2023,11,23240.389,0.032521,0.030095


## Q2
Aggregate stock, bond, and riskless datatables. For each year-month, calculate the lagged market value and excess value-weighted returns for both stocks and bonds. Your output should be from January 1926 to December 2023, at a monthly frequency

In [126]:
Monthly_CRSP_Stock = conn.raw_sql("""
                        select date, VWRETD, totval
                        from crspq.msi
                        """)
Monthly_CRSP_Stock['date']   = pd.DataFrame(Monthly_CRSP_Stock[['date']].values.astype('datetime64[ns]')) + MonthEnd(0)
Monthly_CRSP_Stock = Monthly_CRSP_Stock.rename(columns={"vwretd":"mkt_crsp","totval":"mkt_crsp_mktcap"}).copy()
Monthly_CRSP_Stock['Year'] = Monthly_CRSP_Stock['date'].dt.year
Monthly_CRSP_Stock['Month'] = Monthly_CRSP_Stock['date'].dt.month

In [11]:
Monthly_CRSP_Riskless = conn.raw_sql("""
                      select caldt, t30ret, t90ret
                      from crspq.mcti
                      """)
Monthly_CRSP_Riskless['caldt']   = pd.DataFrame(Monthly_CRSP_Riskless[['caldt']].values.astype('datetime64[ns]')) + MonthEnd(0)
Monthly_CRSP_Riskless = Monthly_CRSP_Riskless.rename(columns={"caldt":"date","t30ret":"rf30","t90ret":"rf90"}).copy()

In [12]:
Monthly_CRSP_Riskless = Monthly_CRSP_Riskless.iloc[1:]

In [13]:
Monthly_CRSP_Riskless['Year'] = Monthly_CRSP_Riskless['date'].dt.year
Monthly_CRSP_Riskless['Month'] = Monthly_CRSP_Riskless['date'].dt.month

In [154]:
def PS2_Q2(Monthly_CRSP_Stocks, Monthly_CRSP_Bonds, Monthly_CRSP_Riskless):
    Monthly_CRSP_Stocks = pd.merge(Monthly_CRSP_Stocks, Monthly_CRSP_Riskless, on = ['Year','Month'], how='outer')
    Monthly_CRSP_Bonds = pd.merge(Monthly_CRSP_Bonds, Monthly_CRSP_Riskless, on = ['Year','Month'], how='outer')
    
    # Calculate excess returns over the riskless rate for stocks and bonds
    Monthly_CRSP_Bonds['Bond_Excess_Vw_Ret'] = Monthly_CRSP_Bonds['Bond_Vw_Ret'] - Monthly_CRSP_Bonds['rf30']
    Monthly_CRSP_Stocks['Stock_Excess_Vw_Ret'] = Monthly_CRSP_Stocks['mkt_crsp'] - Monthly_CRSP_Stocks['rf30']
    Monthly_CRSP_Stocks['Stock_lag_MV'] = Monthly_CRSP_Stocks['mkt_crsp_mktcap'].shift(1) / 1e6
    
    # Combine stock and bond dataframes
    combined_df = pd.merge(Monthly_CRSP_Stocks[['Year','Month', 'Stock_lag_MV', 'Stock_Excess_Vw_Ret']], 
                           Monthly_CRSP_Bonds[['Year','Month', 'Bond_lag_MV', 'Bond_Excess_Vw_Ret']], 
                           on = ['Year','Month'], how='outer')

    return combined_df

Monthly_CRSP_Universe = PS2_Q2(Monthly_CRSP_Stock, Monthly_CRSP_Bonds, Monthly_CRSP_Riskless)
Monthly_CRSP_Universe.iloc[1:].reset_index(drop=True)

,Year,Month,Stock_lag_MV,Stock_Excess_Vw_Ret,Bond_lag_MV,Bond_Excess_Vw_Ret
0,1926,1,27.487487,-0.002390,19.502,0.003878
1,1926,2,27.624241,-0.035814,19.502,0.001076
2,1926,3,26.752064,-0.066780,18.884,0.000810
3,1926,4,25.083173,0.033957,19.231,0.003414
4,1926,5,25.886744,0.011753,19.227,0.002287
...,...,...,...,...,...,...
1171,2023,8,58691.261000,-0.025142,22692.061,-0.008147
1172,2023,9,57387.590300,-0.051657,23294.014,-0.020002
1173,2023,10,54634.658500,-0.033582,23050.525,-0.014929
1174,2023,11,52874.044900,0.087633,23240.389,0.025671


## Q3
Calculate the monthly unlevered and levered risk-parity portfolio returns as defined by Asness, Frazzini, and Pedersen (2012).3 For the levered risk-parity portfolio, match the value-weighted portfolio’s ˆσ over the longest matched holding period of both. Your output should be from January 1926 to December 2023, at a monthly frequenc

In [151]:
def PS2_Q3(Monthly_CRSP_Universe):
    # set the portfolio weight in each asset class equal to the inverse of its volatility, 
    # estimated using 3-year monthly excess returns up to month t − 1,
    Monthly_CRSP_Universe['Stock_inverse_sigma_hat'] = 1 / (Monthly_CRSP_Universe['Stock_Excess_Vw_Ret'].rolling(window=36).std() * np.sqrt(12))
    Monthly_CRSP_Universe['Bond_inverse_sigma_hat'] = 1 / (Monthly_CRSP_Universe['Bond_Excess_Vw_Ret'].rolling(window=36).std() * np.sqrt(12))
    Monthly_CRSP_Universe['Unlevered_k'] = 1 / (Monthly_CRSP_Universe['Stock_inverse_sigma_hat'] + Monthly_CRSP_Universe['Bond_inverse_sigma_hat'])
    Monthly_CRSP_Universe['Stock_weight_unlevered'] = (Monthly_CRSP_Universe['Stock_inverse_sigma_hat'] * Monthly_CRSP_Universe['Unlevered_k']).shift(1)
    Monthly_CRSP_Universe['Bond_weight_unlevered'] = (Monthly_CRSP_Universe['Bond_inverse_sigma_hat'] * Monthly_CRSP_Universe['Unlevered_k']).shift(1)

    # calcultae Excess_Unlevered_RP_Ret
    Monthly_CRSP_Universe['Excess_Unlevered_RP_Ret'] = Monthly_CRSP_Universe['Stock_weight_unlevered'] * Monthly_CRSP_Universe['Stock_Excess_Vw_Ret'] + Monthly_CRSP_Universe['Bond_weight_unlevered'] * Monthly_CRSP_Universe['Bond_Excess_Vw_Ret']
    
    # Value-weighted portfolio return above riskless r
    Monthly_CRSP_Universe['Excess_Vw_Ret'] = (Monthly_CRSP_Universe['Stock_Excess_Vw_Ret'] * Monthly_CRSP_Universe['Stock_lag_MV'] 
                     + Monthly_CRSP_Universe['Bond_Excess_Vw_Ret'] * Monthly_CRSP_Universe['Bond_lag_MV']) / (Monthly_CRSP_Universe['Stock_lag_MV'] + Monthly_CRSP_Universe['Bond_lag_MV'])
    
    # “60-40” is a portfolio that allocates 60% in stocks and 40% in bonds
    Monthly_CRSP_Universe['Excess_60_40_Ret'] = Monthly_CRSP_Universe['Stock_Excess_Vw_Ret'] * 0.6 + Monthly_CRSP_Universe['Bond_Excess_Vw_Ret'] * 0.4
    
    # calculate levered_k to match the sigma of Excess_Vw_Ret
    Monthly_CRSP_Universe['Levered_k'] = Monthly_CRSP_Universe['Excess_Vw_Ret'].std() / Monthly_CRSP_Universe['Excess_Unlevered_RP_Ret'].std()
    
    # calcultae Excess_Unlevered_RP_Ret and Excess_Levered_RP_Ret
    Monthly_CRSP_Universe['Excess_Levered_RP_Ret'] = Monthly_CRSP_Universe['Excess_Unlevered_RP_Ret'] * Monthly_CRSP_Universe['Levered_k']
    
    return Monthly_CRSP_Universe[['Year', 'Month', 'Stock_Excess_Vw_Ret', 'Bond_Excess_Vw_Ret', 'Excess_Vw_Ret', 'Excess_60_40_Ret', 
                           'Stock_inverse_sigma_hat', 'Bond_inverse_sigma_hat', 'Unlevered_k',
             'Excess_Unlevered_RP_Ret', 'Levered_k', 'Excess_Levered_RP_Ret']]

Port_Rets = PS2_Q3(Monthly_CRSP_Universe)
Port_Rets

,Year,Month,Stock_Excess_Vw_Ret,Bond_Excess_Vw_Ret,Excess_Vw_Ret,Excess_60_40_Ret,Stock_inverse_sigma_hat,Bond_inverse_sigma_hat,Unlevered_k,Excess_Unlevered_RP_Ret,Levered_k,Excess_Levered_RP_Ret
0,1925,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.241631,NaN
1,1926,1,-0.002390,0.003878,0.000211,0.000117,NaN,NaN,NaN,NaN,3.241631,NaN
2,1926,2,-0.035814,0.001076,-0.020548,-0.021058,NaN,NaN,NaN,NaN,3.241631,NaN
3,1926,3,-0.066780,0.000810,-0.038812,-0.039744,NaN,NaN,NaN,NaN,3.241631,NaN
4,1926,4,0.033957,0.003414,0.020702,0.021740,NaN,NaN,NaN,NaN,3.241631,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1172,2023,8,-0.025142,-0.008147,-0.020403,-0.018344,5.643009,24.832719,0.032813,-0.011257,3.241631,-0.036492
1173,2023,9,-0.051657,-0.020002,-0.042518,-0.038995,5.592601,24.274484,0.033482,-0.025864,3.241631,-0.083840
1174,2023,10,-0.033582,-0.014929,-0.028047,-0.026121,5.566174,24.023730,0.033795,-0.018422,3.241631,-0.059717
1175,2023,11,0.087633,0.025671,0.068714,0.062848,5.792002,22.222794,0.035695,0.037327,3.241631,0.121000


## Q4
Replicate and report Panel A of Table 2 in Asness, Frazzini, and Pedersen (2012), except for Alpha and t-stat of Alpha columns. Specifically, for all strategies considered, report the annualized average excess returns, t-statistic of the average excess returns, annualized volatility, annualized Sharpe Ratio, skewness, and excess kurtosis. Your sample should be from January 1929 to June 2010, at monthly frequency. Match the format of the table to the extent possible. Discuss the difference between your table and the table reported in the paper. It is zero? If not, justify whether the difference is economically negligible or not. What are the reasons for a nonzero difference?

In [150]:
def PS2_Q4(Port_Rets):
    # Filter the DataFrame for the sample period
    filtered_Port_Rets = Port_Rets[(Port_Rets['Year'] >= 1929) & (Port_Rets['Year'] < 2010) |
                               ((Port_Rets['Year'] == 2010) & (Port_Rets['Month'] <= 6))]

    # Define a function to calculate all required statistics for a single column
    def calculate_statistics(data):
        mean_monthly = data.mean() * 100
        std_monthly = data.std(ddof=1) * 100
        annualized_mean = mean_monthly * 12
        annualized_std = std_monthly * np.sqrt(12)
        sharpe_ratio = annualized_mean / annualized_std if annualized_std != 0 else np.nan
        skewness = stats.skew(data)
        kurtosis = stats.kurtosis(data) # excess kurtosis
        t_stat = stats.ttest_1samp(data, 0).statistic
        
        return {
            'Excess Return': annualized_mean,
            't-stat': t_stat,
            'Volatility': annualized_std,
            'Sharpe Ratio': sharpe_ratio,
            'Skewness': skewness,
            'Excess Kurtosis': kurtosis
        }
    
    # Calculate statistics for each strategy
    stats_list = []
    for strategy in ['Stock_Excess_Vw_Ret', 'Bond_Excess_Vw_Ret', 'Excess_Vw_Ret', 'Excess_60_40_Ret', 'Excess_Unlevered_RP_Ret', 'Excess_Levered_RP_Ret']:
        stats_data = calculate_statistics(filtered_Port_Rets[strategy])
        stats_list.append(stats_data)
    
    # Create the DataFrame with appropriate index and column labels
    stats_df = pd.DataFrame(stats_list, index=['CRSP stocks', 'CRSP bonds', 'Value-weighted portfolio', '60/40 portfolio', 'RP, unlevered', 'RP'])

    return stats_df.round(2)
PS2_Q4(Port_Rets)


,Excess Return,t-stat,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis
CRSP stocks,6.71,3.16,19.15,0.35,0.22,7.80
CRSP bonds,1.36,4.31,2.84,0.48,0.24,4.33
Value-weighted portfolio,3.46,2.59,12.07,0.29,-0.58,4.62
60/40 portfolio,4.57,3.53,11.68,0.39,0.23,7.54
"RP, unlevered",2.03,5.04,3.64,0.56,0.10,2.72
RP,6.60,5.04,11.81,0.56,0.10,2.72
